In [2]:
from numba import njit
import pandas as pd
from pandas.core.interchange.dataframe_protocol import DataFrame
import numpy as np
import scipy.optimize._minimize as minimize

In [ ]:
def find_sigma_i(x_i, x_is, p_i, k, A):
    sig_i = np.linspace(0, 10, 100)
    goal = np.log2(k)
    def sig_exp(x_i, x_is, p_i, sig):
        res = 0
        for i in range(k):
            res += np.exp(-(np.linalg.norm(A[x_i]-A[x_is[i]])-p_i)/sig)
        return res
    

In [27]:
def construct_knn(A: np.ndarray, k=3) -> np.ndarray:
    n_samples = A.shape[0]
    knn_mat = np.zeros((n_samples, k), dtype=int)
    knn_dist = np.zeros((n_samples, 1), dtype=int)
    knn_sigma = np.zeros((n_samples, 1), dtype=int)
    for i in range(n_samples):
        # Compute Euclidean distances
        distances = np.linalg.norm(A - A[i], axis=1)
        # Get sorted indices (excluding the first one, which is the point itself)
        knn_mat[i] = np.argsort(distances)[1:k+1]
        knn_dist[i] = distances[np.argmin(distances)]
    return knn_mat
    

In [ ]:
def 

In [18]:
#input in as a pandas dataframe
def umap(data: pd.DataFrame):
    data = data.copy().to_numpy()
    knn_mat = construct_knn(data)
    return knn_mat

In [28]:
df = pd.read_csv("diabetes.csv")
data = pd.DataFrame({
    'A': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16],
    'B': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16 ],
    'C': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16],
})
print(umap(data))

[[ 1  2  3]
 [ 0  2  3]
 [ 1  3  0]
 [ 2  4  1]
 [ 3  5  2]
 [ 4  6  3]
 [ 5  7  4]
 [ 6  8  5]
 [ 7  9  6]
 [ 8 10  7]
 [ 9 11  8]
 [10 12  9]
 [11 13 10]
 [12 14 11]
 [13 15 12]
 [14 13 12]]
